In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from math import sqrt
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  **Loading the data**

In [2]:
data = pd.read_csv('/kaggle/input/widsdatathon2022/train.csv')
print("The shape of the data is:" + str(data.shape))

In [3]:
test_data = pd.read_csv('/kaggle/input/widsdatathon2022/test.csv')
print("The shape of the test data is:" + str(test_data.shape))

In [4]:
test_data.head()

In [5]:
data.tail(10)

In [6]:
#Let's take a look at all the columns, types and overview of null values.

data.info()

In [7]:
test_data.info()

We can see that we have only 3 columns with object type: 'State_Factor', 'building_class' and 'facility_type'.

# **Exploratory data analysis**

## **Distribution Analysis**

In [8]:
#Let's look at the distribution of our target variable.

sns.displot(
  data=data,
  x="site_eui",
  kind="hist",
  aspect=2,
  bins=30
)

We can see that we have a skewed data. Central tendency is where our general data lies and with machine learning algorithms we are trying to predict general behaviour, so knowing the correct measure of central tendency will help.

In [9]:
#Let's define a function to summarize each numeric variable and also
#look for correlation between such column and the target variable.

def plot_feature(df , col):
    
    plt.figure(figsize = (16,6))
    
    plt.subplot(1,2,1)
        
    if df[col].dtype == 'int64' or df[col].dtype == 'float64' :
        df[col].value_counts().sort_index().plot()
        plt.xticks(rotation=45)
        plt.xlabel(col)
        plt.ylabel('Counts')
    
    else:
        plot = sns.countplot(x=col,data=df)
    
    plt.subplot(1,2,2)
    
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':    
        mean = df.groupby(col)['site_eui'].mean()
        std = df.groupby(col)['site_eui'].std()
        mean.plot()
        plt.fill_between(range(len(std.index)), mean.values-std.values, mean.values + std.values, alpha = 0.1)
        plt.xticks(rotation = 45)
        plt.ylabel('Site EUI') 
        
    else:
        sns.boxplot(x = col, y = 'site_eui', data=df)
    plt.xticks(rotation = 45)
    plt.ylabel('SITE_EUI')
    plt.show()
       

In [10]:
#Let's create a second graph to be able to compare different features.
def compare_features(col1,col2):
    fig = plt.figure(figsize=(8,6))
    sns.scatterplot( x=col1, y=col2, data=data)
    plt.show()

In [11]:
plot_feature(data , 'Year_Factor')

In [12]:
plot_feature(data, 'State_Factor')

In [13]:
plot_feature(data , 'building_class')

In [14]:
plot_feature(data , 'floor_area')

In [15]:
plot_feature(data, 'year_built')

In [16]:
plot_feature(data , 'energy_star_rating')

We can see clearly that there is a negative linear relashionship between energy_star_rating and site_eui. The higher the site_eui, the lower the energy_star_rating.

In [17]:
plot_feature(data , 'ELEVATION')

In [18]:
plot_feature(data , 'avg_temp')

In [19]:
plot_feature(data , 'cooling_degree_days')

In [20]:
plot_feature(data , 'heating_degree_days')

In [21]:
#Now let's compare two features that we think may have a realtion.

compare_features('energy_star_rating', 'year_built')

We can see that there are some buildings with a energy_star_rating between 60 and 80 but they appear as built on year 0, which does not have a lot of sense. We have to do a deeper analysis in year_built variable.

In [22]:
compare_features('energy_star_rating', 'floor_area')

It does not appear to be a relationship between the energy star rating and the building floor area. But we can say that most of our buildings have a floor up to 200000 feet.

There is a very important negative relation between the target variable and energy star rating.

## **Outliers analysis**

In [23]:
#Let's evaluate the categorical columns. 

data.describe(include=['O'])

In [24]:
#Now let's evaluate our numerical features.

data.describe(include=[np.number])

We confirm that we need to evaluate 'year_built' and 'elevation' because both have extreme values compared to the mean. I In the case of 'year_built', even though the mean is 1952, the minimum value is 0 which seems to be a value we need to treat. 

In the case of elevation, the mean is 39 but the max value is 1924.

We also notice that our target variable 'site_eui' has extreme values compared to the rest of the values.

**Outliers in Site_eui**

In [25]:
#First, let's evaluate our target variable.

plt.figure(figsize=(6,6))
sns.boxplot(data=data['site_eui'])
plt.title('Looking for outliers in site_eui feature')
plt.ylabel('Site Eui')

There are several outliers in the 'site_eui' variable so we need to do some research on it, as any changes can be determinant to our model. 
After looking at the Energy Star website we notice that our maximum value of 900 is inside the ranges of certain facility types. For example in hotels, the energy use intensity ranges from less than 100 to more than 800 kbtu/ft2 across all hotels. So our outliers here are a natural part of the population we are studying.

**Dealing with ouliers in year_built**

In [26]:
stat_yearbuilt= data['year_built'].describe()
print(stat_yearbuilt)

In [27]:
IQR = stat_yearbuilt['75%']-stat_yearbuilt['25%']
upper_yb = stat_yearbuilt['75%'] + 1.5*IQR
lower_yb = stat_yearbuilt['25%'] - 1.5*IQR
print('The upper & lower bounds for suspected outliers are {} and {}.'.format(upper_yb,lower_yb))

In [28]:
plt.figure(figsize=(6,6))
sns.boxplot(data=data['year_built'])
plt.title('Looking for outliers in year_built feature')
plt.ylabel('Year built')

In [29]:
#Let's visualize the rows that have values below our lower bound, which is 1852.
data[data.year_built < lower_yb]

We found 101 rows that have outliers below the lower bound. However after searching in the list of the oldest buildings in USA, we found very conserved and still in use buildings built in years between 1600 and 1852.

In [30]:
data[data.year_built < 1650]

Here the oldest building was built in 1600. The rest of outliers are built in year 0 so we can replace them with NaN and treat them later, or we can delete these rows. Due to the small amount of rows with this value, we will delete them. We'll do this for bot train and test datasets.

In [31]:
data = data.drop(data[data.year_built == 0].index) 

**Dealing eith elevation outliers**

In [32]:
stat_elevation= data['ELEVATION'].describe()
print(stat_elevation)

In [33]:
IQR = stat_elevation['75%']-stat_elevation['25%']
upper_e = stat_elevation['75%'] + 1.5*IQR
lower_e = stat_elevation['25%'] - 1.5*IQR
print('The upper & lower bounds for suspected outliers are {} and {}.'.format(upper_e,lower_e))

In [34]:
plt.figure(figsize=(6,6))
sns.boxplot(data=data['ELEVATION'])
plt.title('Looking for outliers in ELEVATION feature')
plt.ylabel('Elevation')

In [35]:
#Let's visualize the rows that have values above the upper bound, which is 88.
data[data.ELEVATION > upper_e]

In [36]:
data.loc[data.ELEVATION > 300,'facility_type'].value_counts()

## **Null value treatment**

In [37]:
data.isnull().sum()

There are 4 columns with > 50% of data missing, so we will proceed to drop them first.

In [38]:
data = data.drop(['direction_max_wind_speed','direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog'], axis=1)
test_data = test_data.drop(['direction_max_wind_speed','direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog'], axis=1)

There are two more columns with null values, which are 'year_built' and 'energy star rating'. 

**Energy star rating:** As saw before in the distribution analysis, energy_star_rating is a very important feature to predict site_eui because it has a negative linear relation with 'site_eui' target variable, so we need to find a way to impute it correctly because it has a lot of null values.

**Year built:** As our missing values in year_built represent only a 2.4% we will drop those rows will null year_built feature.



In [39]:
data = data.dropna(subset = ['year_built'],axis=0)

In [40]:
data.shape

As we have a skewed data we can not consider the mean for imputation. Median would be a right central tendency measure in this case. But what about considering KNN imputation or iterative imputation? We will evaluate this and make the imputation later. 

## **Encoding categorical values**

**Encoding facility_type**

In [41]:
data['facility_type'].value_counts()

We have 60 different categories in facility_type which would make it tedious to apply one_hot_encoding, but we also do not want to apply an ordinal encoding as these facilities do not have an order.

In [42]:
eui_per_facility = data.groupby('facility_type').site_eui.median()

In [43]:
median_dict = eui_per_facility.to_dict()
median_dict

In [44]:
#Now we can use map() function and provide the dictionary as argument to create a new column for encoding our facility types.

data['facility_codes'] = data['facility_type'].map(median_dict)
test_data['facility_codes'] = test_data['facility_type'].map(median_dict)

In [45]:
data.head()

Now that we have our facility_type encoded in a new column, we will proceed to drop our initial facility_type column and now encode our building_class using an ordinal encoder.

In [46]:
data = data.drop(['facility_type'], axis=1)
test_data = test_data.drop(['facility_type'], axis=1)

**Encoding building class and state factor**

In [47]:
data['building_class'].value_counts()

In [48]:
data['State_Factor'].value_counts()

In [49]:
#We will apply Onehotencoding
encoded_df = data[['building_class', 'State_Factor']].copy()
encoded_test_df = test_data[['building_class', 'State_Factor']].copy()

In [50]:
#Applying onehotencoding
encoded_df = pd.get_dummies(encoded_df)
encoded_test_df = pd.get_dummies(encoded_test_df)

#Now we drop the original columns
data = data.drop(['building_class', 'State_Factor'], axis = 1)
test_data = test_data.drop(['building_class', 'State_Factor'], axis = 1)

In [51]:
#We must verify if our encoded dataframes match in number of columns
encoded_test_df.head()

In [52]:
#Theres is one State_factor column missing in test_data so we will create that column with values 0.
encoded_test_df['State_Factor_State_6'] = 0

In [53]:
#Switching new column place in df
col_list = list(encoded_test_df)
col_list[7], col_list[8] = col_list[8], col_list[7]
encoded_test_df.columns = col_list

In [54]:
#Let's concatenate our just created encoded df with the rest of the dataframe.

data = pd.concat([data, encoded_df], axis = 1).copy()
test_data = pd.concat([test_data, encoded_test_df], axis = 1).copy()

#Now it's time to separate our features from our target variable in our train set.

target = data['site_eui']

features = data.drop(['site_eui'], axis = 1)



## **Imputation of null values using Iterative Imputation**

In [55]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state=42)

imputed_features = imputer.fit_transform(features)
imputed_test_features = imputer.fit_transform(test_data)

df_imputed = pd.DataFrame(imputed_features, columns=features.columns)
df_test_imputed = pd.DataFrame(imputed_test_features, columns=test_data.columns)

#in case of a knn imputer:
#from sklearn.impute import KNNImputer
#impute_knn = KNNImputer(n_neighbors=3)
#clean_data = impute_knn.fit_transform(data)

## **Normalizing values**

In [56]:
'''
std = StandardScaler()
scaled_features = std.fit_transform(features)
scaled_test_data = std.fit_transform(test_data)
'''

In [57]:
#scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
#scaled_test_df = pd.DataFrame(scaled_test_data, columns=test_data.columns)

## **Analysing Multicollinearity**

In [58]:

# Correlation between different variables

corr= df_imputed[['Year_Factor','floor_area','year_built','energy_star_rating','ELEVATION', 'cooling_degree_days', 'heating_degree_days','avg_temp']].corr()

# Set up the matplotlib plot configuration

f, ax = plt.subplots(figsize=(12, 10))

# Generate a mask for upper traingle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Configure a custom diverging colormap

cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap

sns.heatmap(corr, annot=True, mask = mask, cmap=cmap)


We can see that 'heating_degree_days' has a strong positive relation with 'ELEVATION' and also a strong negative relation with 'avg_temp'. If an independent variable is highly correlated with 1 or more variables, we say that the variable is multicollinear.
We need to remove 'heating_degree_days' from our dataset as they might mislead our algorithms.

In [59]:
df_imputed = df_imputed.drop(['heating_degree_days'], axis=1)
df_test_imputed = df_test_imputed.drop(['heating_degree_days'], axis=1)

For purposes of making a simple linear regression model we will only use the variable that had a very linear relation with the target variable: energy_star_rating.


In [60]:
simple_lr_data = df_imputed[['energy_star_rating']]
simple_lr_test_data = df_test_imputed[['energy_star_rating']]

# **Modeling**

In [61]:
#Let's split our final train data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(simple_lr_data, target, test_size=0.2, random_state=42)

## **Baseline Model**

In [62]:
#Fitting simple linear regression to the training set
lr = LinearRegression()
lr.fit(X_train, y_train)

In [63]:
#to predict the test result
y_pred = lr.predict(X_test)

#to calculate the score
LinReg = round(lr.score(X_test, y_test), 2)
  
#to calculate the rmse
rmse_lr = sqrt(mean_squared_error(y_test,y_pred))
print('RMSE:', rmse_lr)

In [64]:
#Converting to csv to visualize different models in Matlab
#target_variable.to_csv('target.csv')

In [66]:
#create predictions based on test data
lr_predictions = lr.predict(simple_lr_test_data)    

In [67]:
print(len(lr_predictions))

In [68]:
ids = test_data[['id']]
pred_df  = pd.DataFrame(lr_predictions)
ids = ids.astype('int64')

In [69]:
submission_df = pd.DataFrame({'id': ids['id'], 'site_eui':lr_predictions})
submission_df = submission_df.rename(columns={'0' : 'site_eui'})

In [70]:
submission_df.to_csv('submission1.csv', index=False)

In [71]:
#Save the model 

import pickle

filename = "lr_model.pkl"

pickle.dump(lr, open(filename, 'wb'))